# Load Dependencies and Data

In [3]:
import pandas as pd
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [4]:
data_path = "/content/data_source/"
train_file = "train_preprocess.csv"
test_file = "test_preprocess.csv"

train = pd.read_csv(data_path + train_file)
test = pd.read_csv(data_path + test_file)

In [5]:
train.head()

,sent_A,sent_B,category,label
0,Pada awalnya bangsa Israel hanya terdiri dari ...,Pada awalnya bangsa Yahudi hanya terdiri dari ...,menolak perubahan teks terakhir oleh istimewa ...,NotEntail
1,"Salah satu tekniknya adalah periplus , deskrip...",Bangsa Romawi memberi sumbangan pada pemetaan ...,sejarah geografi,NotEntail
2,Etnolinguistik antropologi adalah ilmu yang me...,Etnolinguistik antropologi adalah ilmu yang me...,definisi anthropologi menurut para ahli,NotEntail
3,"Sekarang , tidak ada yang tahu pasti kapan sej...",Sejarah dimulai pada awal penciptaan manusia .,1 penggantian istilah quot prasejarah quot den...,NotEntail
4,"Soekarno tidak memilih bahasanya sendiri , Jaw...","Dengan memilih Bahasa Melayu , para pejuang ke...",bahasa indonesia,NotEntail


In [8]:
test = test.drop(['Unnamed: 0'], axis=1)
test.head()

,sent_A,sent_B,category,label
0,Asam deoksiribonukleat atau lebih dikenal deng...,"Asam deoksiribonukleat , lebih dikenal dengan ...",kategori,Entail_or_Paraphrase
1,DNA pertama kali berhasil dimurnikan pada tahu...,DNA pertama kali berhasil dimurnikan pada tahu...,sejarah,Entail_or_Paraphrase
2,Rantai tunggal yang diperoleh dari DNA sebelum...,Rantai tunggal yang diperoleh dari DNA sebelum...,replikasi copyedit sembunyikan yang belum jelas,Entail_or_Paraphrase
3,Proses replikasi memerlukan keberadaan protein...,Proses replikasi memerlukan protein atau enzim...,replikasi copyedit sembunyikan yang belum jelas,Entail_or_Paraphrase
4,Proses replikasi diawali dengan pembukaan doub...,Proses replikasi diawali dengan pembukaan unta...,replikasi copyedit sembunyikan yang belum jelas,Entail_or_Paraphrase


## Encode Label
Encode the label for both train and test data


*   Entail_or_Paraphrase as 1
*   NotEntail as 0



In [9]:
train['label'].replace({"Entail_or_Paraphrase": 1, "NotEntail": 0}, inplace=True)
test['label'].replace({"Entail_or_Paraphrase": 1, "NotEntail": 0}, inplace=True)

In [10]:
train.head()

,sent_A,sent_B,category,label
0,Pada awalnya bangsa Israel hanya terdiri dari ...,Pada awalnya bangsa Yahudi hanya terdiri dari ...,menolak perubahan teks terakhir oleh istimewa ...,0
1,"Salah satu tekniknya adalah periplus , deskrip...",Bangsa Romawi memberi sumbangan pada pemetaan ...,sejarah geografi,0
2,Etnolinguistik antropologi adalah ilmu yang me...,Etnolinguistik antropologi adalah ilmu yang me...,definisi anthropologi menurut para ahli,0
3,"Sekarang , tidak ada yang tahu pasti kapan sej...",Sejarah dimulai pada awal penciptaan manusia .,1 penggantian istilah quot prasejarah quot den...,0
4,"Soekarno tidak memilih bahasanya sendiri , Jaw...","Dengan memilih Bahasa Melayu , para pejuang ke...",bahasa indonesia,0


In [11]:
test.head(5)

,sent_A,sent_B,category,label
0,Asam deoksiribonukleat atau lebih dikenal deng...,"Asam deoksiribonukleat , lebih dikenal dengan ...",kategori,1
1,DNA pertama kali berhasil dimurnikan pada tahu...,DNA pertama kali berhasil dimurnikan pada tahu...,sejarah,1
2,Rantai tunggal yang diperoleh dari DNA sebelum...,Rantai tunggal yang diperoleh dari DNA sebelum...,replikasi copyedit sembunyikan yang belum jelas,1
3,Proses replikasi memerlukan keberadaan protein...,Proses replikasi memerlukan protein atau enzim...,replikasi copyedit sembunyikan yang belum jelas,1
4,Proses replikasi diawali dengan pembukaan doub...,Proses replikasi diawali dengan pembukaan unta...,replikasi copyedit sembunyikan yang belum jelas,1


# Preprocessing

The function below shows the step by step of whitespace removal, case folding, punctuation removal, stemming, and stopwords removal for a sentence

In [12]:
def preprocess_sentence_func(sentence, stemmer, stopword):
  # WHITESPACE REMOVAL
  new_sentence = sentence.strip()

  # CASE FOLDING
  new_sentence = new_sentence.lower()

  # PUNCTUATION REMOVAL
  new_sentence = new_sentence.translate(str.maketrans("","", string.punctuation))

  # STEMMING
  new_sentence = stemmer.stem(new_sentence)

  # STOPWORDS REMOVAL
  new_sentence = stopword.remove(new_sentence)

  return new_sentence

The function below run the preprocessing step for a dataframe

In [13]:
def preprocess_func(df):
  n = df.shape[0]
  value_arr = []

  # preparation for stemming
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()

  # preparation for stopwords removal
  stop_factory = StopWordRemoverFactory()
  stopword = stop_factory.create_stop_word_remover()
  
  for row in range(n):
    new_row = []
    premis = df['sent_A'][row]
    hipotesis = df['sent_B'][row]

    new_premis = preprocess_sentence_func(premis, stemmer, stopword)
    new_hipotesis = preprocess_sentence_func(hipotesis, stemmer, stopword)

    new_row.append(new_premis)
    new_row.append(new_hipotesis)

    value_arr.append(new_row)
    if row == 0:
      print("running...")
    if row == n-1:
      print("finish")
  
  new_df = pd.DataFrame(data=value_arr, columns=['sent_A', 'sent_B'])
  return new_df

In [14]:
# call function for each dataframe (train and test)
preprocessed_train = preprocess_func(train)
preprocessed_test = preprocess_func(test)

running...
finish
running...
finish


## Preprocessed Data Preparation

In [15]:
train['preprocessed_A'] = preprocessed_train['sent_A']
train['preprocessed_B'] = preprocessed_train['sent_B']

test['preprocessed_A'] = preprocessed_test['sent_A']
test['preprocessed_B'] = preprocessed_test['sent_B']

In [16]:
train.head()

,sent_A,sent_B,category,label,preprocessed_A,preprocessed_B
0,Pada awalnya bangsa Israel hanya terdiri dari ...,Pada awalnya bangsa Yahudi hanya terdiri dari ...,menolak perubahan teks terakhir oleh istimewa ...,0,awal bangsa israel diri satu kelompok keluarga...,awal bangsa yahudi diri satu kelompok keluarga...
1,"Salah satu tekniknya adalah periplus , deskrip...",Bangsa Romawi memberi sumbangan pada pemetaan ...,sejarah geografi,0,salah satu teknik periplus deskripsi labuh dar...,bangsa romawi beri sumbang meta mereka banyak ...
2,Etnolinguistik antropologi adalah ilmu yang me...,Etnolinguistik antropologi adalah ilmu yang me...,definisi anthropologi menurut para ahli,0,etnolinguistik antropologi ilmu ajar lukis cir...,etnolinguistik antropologi ilmu ajar sukusuku ...
3,"Sekarang , tidak ada yang tahu pasti kapan sej...",Sejarah dimulai pada awal penciptaan manusia .,1 penggantian istilah quot prasejarah quot den...,0,sekarang ada tahu kapan sejarah mulai,sejarah mulai awal cipta manusia
4,"Soekarno tidak memilih bahasanya sendiri , Jaw...","Dengan memilih Bahasa Melayu , para pejuang ke...",bahasa indonesia,0,soekarno pilih bahasa sendiri jawa benar bahas...,pilih bahasa melayu juang merdeka satu seperti...


In [17]:
test.head()

,sent_A,sent_B,category,label,preprocessed_A,preprocessed_B
0,Asam deoksiribonukleat atau lebih dikenal deng...,"Asam deoksiribonukleat , lebih dikenal dengan ...",kategori,1,asam deoksiribonukleat lebih kenal singkat dna...,asam deoksiribonukleat lebih kenal dna bahasa ...
1,DNA pertama kali berhasil dimurnikan pada tahu...,DNA pertama kali berhasil dimurnikan pada tahu...,sejarah,1,dna pertama kali hasil murni tahun 1868 ilmuwa...,dna pertama kali hasil murni tahun 1868 ilmuwa...
2,Rantai tunggal yang diperoleh dari DNA sebelum...,Rantai tunggal yang diperoleh dari DNA sebelum...,replikasi copyedit sembunyikan yang belum jelas,1,rantai tunggal oleh dna sebut tindak bagai cet...,rantai tunggal oleh dna sebut tindak bagai cet...
3,Proses replikasi memerlukan keberadaan protein...,Proses replikasi memerlukan protein atau enzim...,replikasi copyedit sembunyikan yang belum jelas,1,proses replikasi perlu protein enzim bantu sal...,proses replikasi perlu protein enzim bantu sal...
4,Proses replikasi diawali dengan pembukaan doub...,Proses replikasi diawali dengan pembukaan unta...,replikasi copyedit sembunyikan yang belum jelas,1,proses replikasi awal buka double helix point ...,proses replikasi awal buka untai ganda dna tit...


## Export Data

In [18]:
result_path = "/content/data_source/preprocessed_result/"
train_result = "preprocessed_train.csv"
test_result = "preprocessed_test.csv"

train.to_csv(result_path + train_result)
test.to_csv(result_path + test_result)